# 1. Setup

In [1]:
!pip install -r requirements.txt

  Using cached astor-0.8.0-py2.py3-none-any.whl (27 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached beautifulsoup4-4.9.1-py3-none-any.whl (115 kB)
  Using cached blinker-1.4.tar.gz (111 kB)
  Using cached brotlipy-0.7.0-cp37-cp37m-win_amd64.whl (372 kB)
Processing c:\users\anany\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca\bs4-0.0.1-py3-none-any.whl
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached cryptography-2.9.2-cp37-cp37m-win_amd64.whl (1.5 MB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Using cached gast-0.2.2.tar.gz (10 kB)
  Using cached grpcio-1.27.2-cp37-cp37m-win_amd64.whl (1.9 MB)
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
Processing c:\tmp\build\80754af9\keras-applications_1594366238411\work


ERROR: Could not install packages due to an EnvironmentError: [Errno 2] No such file or directory: 'C:\\tmp\\build\\80754af9\\keras-applications_1594366238411\\work'



In [2]:
# my gereneral imports
import os
import wget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import time
# url data reader imports
from bs4 import BeautifulSoup
import requests

from zipfile import ZipFile

# Scikit-Learn imports
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# To plot pretty figures
import matplotlib.pyplot as plt
%matplotlib inline

PROJECT_ROOT_DIR = "."
PROJECT = "Bank"
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data", PROJECT)
os.makedirs(DATA_PATH, exist_ok=True)
OUT_PATH = os.path.join(PROJECT_ROOT_DIR, "output", PROJECT)
os.makedirs(OUT_PATH, exist_ok=True)

In [3]:
t0 = time.time()

# to make this notebook's output stable across runs
np.random.seed(42)

# Q3. Modelling Assessment

Build 2 binary classification models using any 2 of the following methods (in R or Python):
1. Logistic Regression
2. Random Forest
3. GBM
4. Xgboost
5. Neural Network

## 3.1 Download the Data

In [4]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00222/"
ext = 'zip'

def list_files(url, ext=''):
    page = requests.get(url).text
#     print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

for fileurl in list_files(url, ext):
#     print(fileurl)
    filename = fileurl.split('/')[-1]
#     print(filename)
    wget.download(fileurl, out=os.path.join(DATA_PATH, filename))

100% [............................................................................] 579043 / 579043

In [5]:
os.listdir(DATA_PATH)

['bank-additional.zip', 'bank.zip']

### 3.1.1 Read the Data

The description of the data states that there are four datasets.
We need
> (1) `bank-additional-full.csv` with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014].[<sup>1</sup>](#fn1)

An examination of the archive `bank-additional.zip` shows that it contains a folder called `bank-additional` with the files we need.

<span id="fn1"><sup>1</sup>
    S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014.
</span>

In [6]:
zip_file = ZipFile(os.path.join(DATA_PATH, 'bank-additional.zip'))
df = pd.read_csv(
    zip_file.open('bank-additional/bank-additional-full.csv'),
    sep=';',
    quotechar='"',
)
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## 3.2 Variable and Data Set Definitions

Let's define output variable and split the data into train, dev, and test set.

In [7]:
print(df['y'].value_counts())

no     36548
yes     4640
Name: y, dtype: int64


In [8]:
y = df['y']
X = df.drop(columns='y')

In [9]:
print(len(y))

41188


We see that the target variable is *yes* for approximately 11% of cases. If we simply split the data into 80-10-10% train/val/test sets, there is a high chance that we end up with a lot of negative examples in the dev and test sets. This, in turn, might bias our evaluation of model performance.

So, there are two options, I think:
1. to use stratified shuffling to ensure that the proportions of positive and negative examples are similar in the split data sets, or
2. use the test set from the website:
    > (2) `bank-additional.csv` with 10% of the examples (4119), randomly selected from (1), and 20 inputs.

I would personally prefer option 1.
But for the sake of completeness, let's see what distribution of the target variable is in the test file from the website.

In [10]:
df_test_web = pd.read_csv(
    zip_file.open('bank-additional/bank-additional.csv'),
    sep=';',
    quotechar='"',
)
print(df_test_web['y'].value_counts())

no     3668
yes     451
Name: y, dtype: int64


It is clear now, that the file on the website was generated using stratified shuffling.

**Note:** In the PySpark exercise, the data was also split into train and test set. However, in this exercise, we are explicitly told that the test data is a subsample of the full data set.

Therefore, I will use stratified shuffling to split the data into train/dev/test sets.

In [11]:
print(X.shape)

(41188, 20)


In [12]:
print(X.dtypes)

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
dtype: object


In [13]:
# # I put this import into the Setup section at the beginning
# from sklearn.model_selection import train_test_split

In [14]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=42)

In [15]:
y_test.value_counts()

no     3655
yes     464
Name: y, dtype: int64

In [16]:
y_trainval.value_counts()

no     32893
yes     4176
Name: y, dtype: int64

In [17]:
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, stratify=y_trainval, test_size=0.1, random_state=42)

Now, each of the train, dev, and test sets have roughly 11% of the positive outcomes.

**Note:** such splitting method ensures that we avoid data mismatch problem, i.e. make our dev and test sets come from similar distributions.

Let's convert our dependent variables in all the sets into integers.

In [18]:
# yesno_dict = {'yes': 1, 'no': 0}
# y_trainval = y_trainval.map(yesno_dict).astype('int32')
# y_train = y_train.map(yesno_dict).astype('int32')
# y_val = y_val.map(yesno_dict).astype('int32')
# y_test = y_test.map(yesno_dict).astype('int32')

In [19]:
# an alternative way to transform labels:

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_trainval = le.fit_transform(y_trainval.astype(str))
y_train = le.transform(y_train.astype(str))
y_val = le.transform(y_val.astype(str))
y_test = le.transform(y_test.astype(str))

## 3.3 Preprocessing Pipeline

### 3.3.1 Data Exploration

Let's first explore the data to see what variables are continuous, what are needed to be categorized, and what are to be transformed into dummy variables (one-hot encoded).

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33362 entries, 22373 to 31364
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             33362 non-null  int64  
 1   job             33362 non-null  object 
 2   marital         33362 non-null  object 
 3   education       33362 non-null  object 
 4   default         33362 non-null  object 
 5   housing         33362 non-null  object 
 6   loan            33362 non-null  object 
 7   contact         33362 non-null  object 
 8   month           33362 non-null  object 
 9   day_of_week     33362 non-null  object 
 10  duration        33362 non-null  int64  
 11  campaign        33362 non-null  int64  
 12  pdays           33362 non-null  int64  
 13  previous        33362 non-null  int64  
 14  poutcome        33362 non-null  object 
 15  emp.var.rate    33362 non-null  float64
 16  cons.price.idx  33362 non-null  float64
 17  cons.conf.idx   33362 non-n

We clearly see that the following variables are categorical (or binary):
* job,
* marital,
* education,
* housing,
* loan,
* contact,
* month,
* day_of_week,
* poutcome.

Let's check what values they take.

In [21]:
for col in X_train.select_dtypes(exclude=['int64', 'float64']).columns:
    print(col)
    print(X_train[col].value_counts())
    print("\n")

job
admin.           8415
blue-collar      7510
technician       5443
services         3227
management       2386
retired          1383
entrepreneur     1187
self-employed    1142
housemaid         867
unemployed        823
student           720
unknown           259
Name: job, dtype: int64


marital
married     20220
single       9357
divorced     3721
unknown        64
Name: marital, dtype: int64


education
university.degree      9845
high.school            7711
basic.9y               4887
professional.course    4231
basic.4y               3372
basic.6y               1869
unknown                1430
illiterate               17
Name: education, dtype: int64


default
no         26415
unknown     6944
yes            3
Name: default, dtype: int64


housing
yes        17503
no         15040
unknown      819
Name: housing, dtype: int64


loan
no         27452
yes         5091
unknown      819
Name: loan, dtype: int64


contact
cellular     21145
telephone    12217
Name: contact, dtype: i

In [22]:
# a list of columns to be transformed from string to categorical numeric objects
str_cat_vars = X_train.select_dtypes(exclude=['int64', 'float64']).columns.values
str_cat_vars

array(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'poutcome'], dtype=object)

In [23]:
# a list of numeric columns to be scaled
num_vars = X_train.select_dtypes(include=['int64', 'float64']).columns.values
num_vars

array(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype=object)

**NOTE:** The description of the data says:
> DURATION: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

Following this recommendation, I exclude the duration variable from the analysis.

In [24]:
try:
    num_vars = np.setdiff1d(num_vars, ['duration'])
except ValueError:
    print("`num_vars` does not contain 'duration'")

### 3.3.2 Declare the Pipeline

I want the following steps to be processed in my preprocessing pipeline.
1. Transform text variables into categorical numeric;
2. Transform categorical variables into a set of binary variables (dummies, or one-hot encoded);
3. Standardize continuous variables.

**Note:** I treat variables of type `int64` as continuous. Sometimes, we want to treat such variables as categorical variables, so we then transform them into a set of dummy variables.

In [25]:
# # I put these imports into the Setup section at the beginning
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# encoder = OrdinalEncoder()
# dummies = OneHotEncoder()

dummy_pipeline = Pipeline([
        ('encoder', OrdinalEncoder()),
        ('dummies', OneHotEncoder()),
    ])

# scaler = StandardScaler()

prep_pipeline = ColumnTransformer([
        ("cat", dummy_pipeline, str_cat_vars),
        ("num", StandardScaler(), num_vars),
    ])

In [26]:
X_train_prepared = prep_pipeline.fit_transform(X_train)

In [27]:
X_train_prepared

array([[ 0.        ,  0.        ,  0.        , ...,  0.84745082,
         0.19474007, -0.34890404],
       [ 0.        ,  0.        ,  0.        , ...,  0.84745082,
         0.19474007, -0.34890404],
       [ 1.        ,  0.        ,  0.        , ..., -0.9393463 ,
         0.19474007, -0.34890404],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.33357351,
         0.19474007, -0.34890404],
       [ 0.        ,  0.        ,  0.        , ..., -0.9393463 ,
         0.19474007, -0.34890404],
       [ 0.        ,  0.        ,  0.        , ..., -0.9393463 ,
         0.19474007, -0.34890404]])

In [28]:
X_val_prepared = prep_pipeline.transform(X_val)  # we do not *fit* the test data!
X_test_prepared = prep_pipeline.transform(X_test)  # we do not *fit* the test data!

## 3.4 Build a Model

The task is to build 2 binary classification models using any 2 of the following methods:
1. Logistic Regression,
2. Random Forest,
3. GBM,
4. Xgboost,
5. Neural Network.

Let's train the first four and see their performance and keep Nerual Nets as a cherry on a cake! :)

**Note:** I train the basic model in this section. I will tune hyperparameters later.

### 3.4.1 Logistic Regression

In [29]:
from sklearn.linear_model import LogisticRegression

I change the maximum number of iterations taken for the solvers to converge (default is 100 which is too small).

In [30]:
log_reg = LogisticRegression(solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_prepared, y_train)

LogisticRegression(max_iter=5000, random_state=42)

How well we do on the train set

In [31]:
log_reg.score(X_train_prepared, y_train)

0.8996762784005755

How well we do on the validation set

In [32]:
log_reg.score(X_val_prepared, y_val)

0.907742109522525

I leave model evaluation assessment on the test set for later analysis after I fine-tune the model with hyperparameters tuning.

### 3.4.2 Random Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train_prepared, y_train)

RandomForestClassifier(random_state=42)

In [35]:
forest_clf.score(X_train_prepared, y_train)

0.9953240213416462

In [36]:
forest_clf.score(X_val_prepared, y_val)

0.8972214728891287

### 3.4.3 GBM

In [37]:
from sklearn.ensemble import GradientBoostingClassifier

In [38]:
gb_clf = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_clf.fit(X_train_prepared, y_train)

GradientBoostingClassifier(random_state=42)

In [39]:
gb_clf.score(X_train_prepared, y_train)

0.9048018703914633

In [40]:
gb_clf.score(X_val_prepared, y_val)

0.9088211491772322

### 3.4.4 XGBoost

In [41]:
from xgboost import XGBClassifier

**WARNING.** XGBClassifier gives errors when printing the output (after the `fit` method?..). It seems to be an issue with Scikit-Learn version 0.23.1 and higher, see discussion here: https://github.com/dmlc/xgboost/issues/5668

I updated scikit-learn to version 0.23.2 and it still appears as an error...

In [42]:
xgb_clf = XGBClassifier(random_state=42)
xgb_clf.fit(X_train_prepared, y_train)

KeyError: 'base_score'

KeyError: 'base_score'

In [43]:
xgb_clf.score(X_train_prepared, y_train)

0.9294406810143276

In [44]:
xgb_clf.score(X_val_prepared, y_val)

0.906123550040464

### 3.4.5 Preliminary Summary

It is clear that Random Forest and Gradient Boosting classifiers overfit on the training set, Logistic Regression does not overfit on the training set, and XGBoost classifier overfits slightly.
All four classifiers show similar performance on the validation set: ~90% accuracy.
Therefore, we have to do some regularization and hyperparameter tuning, as well as some feature engineering.

## 3.5 Improving the Models

### 3.5.1 Feature Engineering

First, let's try to come up with some new features that may better explain the outcome variable.
There are few general ways to generate new features.
1. Adding polynomial features.
2. Categorizing continuous variables;
3. Combining the existing features to produce new, meaningful ones;

The first option is used when we do not want to give a thought about what features we want to use. This option might be useful when we underfit the data (not our case). Another use of this way is to produce interactions between features.

The second option is used when we believe that it is not the value that is important but the groups that are defined by certain levels.
For example, suppose we new income levels of the respondents.
We could use categorical income variable with 3 categories: 'low', 'medium', and 'high', because we might expect similar behavior of respondents in the same income group but different across those groups.

The third option requires the knowledge of the problem at hand.
Were I a banking specialist, I could design new features like "the ratio of the number of times the client was contacted during last campaign to the number of times the client was contacted during current campaign".

For demonstration purposes, I do all three steps below.

In [45]:
num_vars

array(['age', 'campaign', 'cons.conf.idx', 'cons.price.idx',
       'emp.var.rate', 'euribor3m', 'nr.employed', 'pdays', 'previous'],
      dtype=object)

#### Manually combine existing features

To add features created manually to the pipeline, we need to define a class that returns self on fit (does nothing) and returns a numpy array with additional feature.

**Note:** This class operates on numerical features only, so we will construct an additional pipeline to deal with all numeric features at once.

In [46]:
from sklearn.base import BaseEstimator, TransformerMixin

"""
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
"""

current_c_ix, past_c_ix = 1, 8

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_num_contacts_ratio = True):  # no *args or **kargs
        self.add_num_contacts_ratio = add_num_contacts_ratio
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        if self.add_num_contacts_ratio:
            num_contacts_ratio = X[:, past_c_ix] / X[:, current_c_ix]
            return np.c_[X, num_contacts_ratio]
        else:
            return X

In [47]:
adder = CombinedAttributesAdder()

#### Categorize continuous variables

In [48]:
pd.crosstab(y_train, X_train['age'], margins = False)

age,17,18,19,20,21,22,23,24,25,26,...,83,84,85,86,87,88,89,91,92,98
row_0,,,,,,,,,,,,,,,,,,,,,
0,2,16,18,34,61,76,142,313,401,472,...,5,4,5,1,0,8,0,2,0,0
1,2,7,16,20,22,30,38,73,75,102,...,5,3,5,4,1,9,2,0,3,1


We can see that the clients' age varies a lot. Let's build a dummy variable that represents that the person has not yet reached the retirement age (66.5 years in Portugal).

In [49]:
from sklearn.preprocessing import Binarizer

In [50]:
binarizer = Binarizer(threshold=66.5)

We have to keep in mind that if we replace `age` variable with a binary variable defined above, we need to exclude `age` from numeric variables `num_vars`.

In [51]:
binary_vars = ['age',]

new_num_vars = np.setdiff1d(num_vars, binary_vars)

**Note:** we could also use an alternative way - use `KBinsDiscretizer` to bin continuous data into intervals.

#### Add quadratic features (polynomial with $n=2$)

In [52]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)

### 3.5.2 Build a New Pipeline

In [53]:
# for numeric features
col_transformer = ColumnTransformer([
        ("categorizer", dummy_pipeline, str_cat_vars),
        ("binarizer", Binarizer(threshold=66.5), binary_vars),
        ("scaler", StandardScaler(), new_num_vars),
    ])

full_pipeline = Pipeline([
    ('col_transformer', col_transformer),
    ("poly_features", PolynomialFeatures(degree=2, include_bias=False)),
])

In [54]:
X_train_complete = full_pipeline.fit_transform(X_train)
X_val_complete = full_pipeline.transform(X_val)
X_test_complete = full_pipeline.transform(X_test)

In [55]:
X_train.shape, X_train_complete.shape

((33362, 20), (33362, 2015))

In [56]:
t_lr = time.time()
log_reg.fit(X_train_complete, y_train)
print(f"Logistic regression trained in {time.time() - t_lr}s")
print(f"Train accuracy: {log_reg.score(X_train_complete, y_train)}, val accuracy: {log_reg.score(X_val_complete, y_val)}")

Logistic regression trained in 103.41946268081665s
Train accuracy: 0.9082489059408908, val accuracy: 0.9018073914216348


In [57]:
t_rf = time.time()
forest_clf.fit(X_train_complete, y_train)
print(f"Random forest classifier trained in {time.time() - t_rf}s")
print(f"Train accuracy: {forest_clf.score(X_train_complete, y_train)}, val accuracy: {forest_clf.score(X_val_complete, y_val)}")

Random forest classifier trained in 40.053128242492676s
Train accuracy: 0.9843534560278161, val accuracy: 0.8893984353925006


In [58]:
t_gb = time.time()

gb_clf.fit(X_train_complete, y_train)
print(f"Gradient boosting classifier trained in {time.time() - t_gb}s")
print(f"Train accuracy: {gb_clf.score(X_train_complete, y_train)}, val accuracy: {gb_clf.score(X_val_complete, y_val)}")

Gradient boosting classifier trained in 353.8548309803009s
Train accuracy: 0.9094478748276482, val accuracy: 0.9066630698678176


In [59]:
t_xgb = time.time()
xgb_clf.fit(X_train_complete, y_train)
print(f"XGBoost classifier trained in {time.time() - t_xgb}s")
print(f"Train accuracy: {xgb_clf.score(X_train_complete, y_train)}, val accuracy: {xgb_clf.score(X_val_complete, y_val)}")

XGBoost classifier trained in 187.3780586719513s
Train accuracy: 0.9302499850128889, val accuracy: 0.9009981116806043


It takes quite some time to train a GB Classifier on the dataset with 30k instances and 2k features (over 5 minutes on my laptop with i5-7200 CPU 2.5GHz)...

Also note that the performance has not improved, most likely because we replaced age variable with a dummy variable that is unlikely to produce meaningful predictions (I did that just to play around with the API).

It seems that Random Forest and XGBoost Classifiers perform best (although RF classifier overfits the training set).
I will tune these two models in the subsequent analysis.

Furthermore, I revert binarizing `age` variable but would like to add polynomial features of degree 2.

In [60]:
final_pipeline = Pipeline([
    ('basic_prep', prep_pipeline),
    ("poly_features", PolynomialFeatures(degree=2, include_bias=True)),
])

X_train_final = final_pipeline.fit_transform(X_train)
X_val_final = final_pipeline.transform(X_val)
X_test_final = final_pipeline.transform(X_test)

t_rf = time.time()
forest_clf.fit(X_train_final, y_train)
print(f"Random forest classifier trained in {time.time() - t_rf}s on polyfeatures (d=2)")
print(f"Train accuracy: {forest_clf.score(X_train_final, y_train)}, val accuracy: {forest_clf.score(X_val_final, y_val)}")

t_xgb = time.time()
xgb_clf.fit(X_train_final, y_train)
print(f"XGBoost classifier trained in {time.time() - t_xgb}s on polyfeatures (d=2)")
print(f"Train accuracy: {xgb_clf.score(X_train_final, y_train)}, val accuracy: {xgb_clf.score(X_val_final, y_val)}")

Random forest classifier trained in 40.32072067260742s on polyfeatures (d=2)
Train accuracy: 0.9953240213416462, val accuracy: 0.8985702724575129
XGBoost classifier trained in 182.69800925254822s on polyfeatures (d=2)
Train accuracy: 0.9351058090042563, val accuracy: 0.9004585918532506


Well, well, well... It seems that adding polynomial features did not help at all but slowed down my algorithm.
The best approach is the simplest approach, right?..

### 3.5.3 Regularization and Hyperparameter Tuning

**WARNING:** *Please, keep in mind that the cross-validation methods presented below might take much longer time to run on your local machine!.*

There are two classes in `Scikit-Learn` that help us tune hyperparameter:
1. `GridSearchCV`, and
2. `RandomizedSearchCV`.

Let's try both of them.

In [61]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

**NOTE:** I will *not* try grid search cross-validation for the degrees of polynomial features or other pre-processing steps.

#### Grid Search Cross-Validation for Random Forest Classifier

In [62]:
rf_clf = RandomForestClassifier(random_state=42)

rf_param_grid = [
    {'n_estimators': [10, 50, 100, 150, 200], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2],},
]

# Note that we ask GridSearchCV to randomly split the data into train and val sets 5 times
# GridSearchCV automatically uses StratifiedKFold splitter
# if the estimator is a classifier and y is either binary or multiclass
rf_grid_search = GridSearchCV(rf_clf, rf_param_grid, cv=5, return_train_score=True, n_jobs=-1)

# because of that splitter we can use the whole train+val set to feed to GridSearchCV
X_trainval_prepared = prep_pipeline.transform(X_trainval)  # let's use the scaler we fit on the *train* set

t_rf_GridSearchCV = time.time()
rf_grid_search.fit(X_trainval_prepared, y_trainval)
print(f"GridSearchCV for RF classifier trained in {time.time() - t_rf_GridSearchCV}s on 4 cores")

GridSearchCV for RF classifier trained in 345.16774129867554s on 4 cores


In [63]:
rf_grid_search.best_params_

{'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}

In [64]:
rf_grid_search.best_estimator_

RandomForestClassifier(min_samples_leaf=2, min_samples_split=10,
                       n_estimators=200, random_state=42)

In [65]:
rf_grid_search.best_estimator_.score(X_trainval_prepared, y_trainval)

0.9261377431276808

In [66]:
rf_grid_search.best_estimator_.score(X_test_prepared, y_test)

0.8997329448895363

Not bad. But also not an outstanding result. We clearly avoid overfitting (compared to the first prototypes).

#### Randomized Search Cross-Validation for XGBoost Classifier

I will not create a learning rate schedule. Instead, I will use randomized search to tune the LR and the penalty parameter for L2 regularization (`reg_lambda = 1` by default in XGBClassifier with the default `booster = gbtree`).

In [67]:
from scipy.stats import expon, reciprocal

xgb_clf = XGBClassifier(random_state=42)

xgb_param_dist = {
    'learning_rate': reciprocal(0.01, 10),
    'reg_lambda': expon(scale=1.),
}

xgb_randomized_search = RandomizedSearchCV(xgb_clf, param_distributions=xgb_param_dist,
                                           n_iter=50, cv=5, verbose=2, random_state=42,
                                           n_jobs=-1)

t_xgb_RandomizedSearchCV = time.time()
xgb_randomized_search.fit(X_trainval_prepared, y_trainval)
print(f"RandomizedSearchCV for XGBoost classifier trained in {time.time() - t_xgb_RandomizedSearchCV}s on 4 cores")

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 20.7min finished


RandomizedSearchCV for XGBoost classifier trained in 1249.4742312431335s on 4 cores


In [68]:
xgb_randomized_search.best_params_

{'learning_rate': 0.010388823104027934, 'reg_lambda': 1.689896777486126}

In [69]:
xgb_randomized_search.best_estimator_

KeyError: 'base_score'

KeyError: 'base_score'

A-ha! Same error as above, when printing out the estimator!

In [70]:
xgb_randomized_search.best_estimator_.score(X_trainval_prepared, y_trainval)

0.9064177614718498

In [71]:
xgb_randomized_search.best_estimator_.score(X_test_prepared, y_test)

0.8999757222626851

Well... We clearly avoided overfitting. BUT we did not get much imporvement either (compared to the first prototypes with the default parameters).

Let's build a soft voting classifier on top of these two and, maybe, try an SVM Classifier (should work fine as we have less than 100k instances and 20 features).

###  Soft Voting Classifier

In [72]:
xgb_randomized_search.best_params_

{'learning_rate': 0.010388823104027934, 'reg_lambda': 1.689896777486126}

In [73]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

rxgb_clf = XGBClassifier(**xgb_randomized_search.best_params_, random_state=42)
rrf_clf = RandomForestClassifier(**rf_grid_search.best_params_, random_state=42)
# I was wrong, it takes some enormous time to train SVM classifier with this number of instances...
# svm_clf = SVC(gamma="scale", probability=True, random_state=42)
# I will not train or use SVC in subsequent analysis or in the voting classifier, as I planned initially.

voting_clf = VotingClassifier(
    estimators=[('xgb', rxgb_clf), ('rf', rrf_clf)],
    voting="soft")

In [74]:
from sklearn.metrics import accuracy_score

for clf in (rxgb_clf, rf_clf, voting_clf):
    clf.fit(X_train_prepared, y_train)
    y_pred = clf.predict(X_test_prepared)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

XGBClassifier 0.8982762806506434
RandomForestClassifier 0.8926924010682205
VotingClassifier 0.8997329448895363


Soft Voting Classifier did not provide much better results (althouth, the accuracy on the test set is 0.15% higher than for XGBClassifier).

## 3.6 Neural Network

In [75]:
np.random.seed(42)
tf.random.set_seed(42)

In [76]:
from tensorflow import keras

keras.backend.clear_session()

In [77]:
tf.__version__

'2.2.0'

In [78]:
keras.__version__

'2.3.0-tf'

### 3.6.1 A Simple NN

In [79]:
model = keras.models.Sequential([
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [80]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.SGD(lr=.01),
              metrics=["accuracy"])

In [81]:
history = model.fit(
    X_train_prepared,
    y_train,
    epochs=100,
    validation_data=(X_val_prepared, y_val)
)

Epoch 1/100
1043/1043 [==============================] - 2s 2ms/step - loss: 0.3064 - accuracy: 0.8942 - val_loss: 0.2817 - val_accuracy: 0.9029
Epoch 2/100
1043/1043 [==============================] - 1s 1ms/step - loss: 0.2816 - accuracy: 0.8992 - val_loss: 0.2790 - val_accuracy: 0.9013
Epoch 3/100
1043/1043 [==============================] - 1s 1ms/step - loss: 0.2788 - accuracy: 0.8998 - val_loss: 0.2770 - val_accuracy: 0.9034
Epoch 4/100
1043/1043 [==============================] - 1s 1ms/step - loss: 0.2773 - accuracy: 0.9005 - val_loss: 0.2762 - val_accuracy: 0.9040
Epoch 5/100
1043/1043 [==============================] - 2s 1ms/step - loss: 0.2761 - accuracy: 0.9004 - val_loss: 0.2761 - val_accuracy: 0.9032
Epoch 6/100
1043/1043 [==============================] - 2s 1ms/step - loss: 0.2753 - accuracy: 0.9010 - val_loss: 0.2750 - val_accuracy: 0.9034
Epoch 7/100
1043/1043 [==============================] - 1s 1ms/step - loss: 0.2745 - accuracy: 0.9012 - val_loss: 0.2753 - val_ac

1043/1043 [==============================] - 2s 1ms/step - loss: 0.2558 - accuracy: 0.9060 - val_loss: 0.2734 - val_accuracy: 0.9037
Epoch 58/100
1043/1043 [==============================] - 2s 2ms/step - loss: 0.2551 - accuracy: 0.9075 - val_loss: 0.2740 - val_accuracy: 0.9048
Epoch 59/100
1043/1043 [==============================] - 2s 2ms/step - loss: 0.2547 - accuracy: 0.9070 - val_loss: 0.2750 - val_accuracy: 0.9029
Epoch 60/100
1043/1043 [==============================] - 2s 2ms/step - loss: 0.2544 - accuracy: 0.9077 - val_loss: 0.2740 - val_accuracy: 0.9050
Epoch 61/100
1043/1043 [==============================] - 2s 2ms/step - loss: 0.2540 - accuracy: 0.9076 - val_loss: 0.2745 - val_accuracy: 0.9045
Epoch 62/100
1043/1043 [==============================] - 2s 1ms/step - loss: 0.2537 - accuracy: 0.9079 - val_loss: 0.2734 - val_accuracy: 0.9042
Epoch 63/100
1043/1043 [==============================] - 2s 1ms/step - loss: 0.2531 - accuracy: 0.9079 - val_loss: 0.2745 - val_accuracy

In [82]:
y_pred_nn = model.predict(X_test_prepared)
y_pred_nn

array([[0.1217562 ],
       [0.34475255],
       [0.94861007],
       ...,
       [0.12860693],
       [0.00728785],
       [0.01481945]], dtype=float32)

In [83]:
accuracy_score(y_test, (y_pred_nn>=.5))

0.8953629521728574

It seems that such a simple NN does not provide a better forecast than a Soft Voting Classifier.
Perhaps, using polynomial features and/or a deeper network would help?

### 3.6.2 Deeper NN with Optimal Stopping, Dropout Regularization, and LR Schedule

In [84]:
keras.backend.clear_session()

In [85]:
deeper_model_1 = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(1, activation="sigmoid")
])

In [86]:
# use keras scheduler callback
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# use early stopping callback
early_stopping = keras.callbacks.EarlyStopping(patience=10)
# I will *not* use tensorboard

# NAdam is RMSprop with Nesterov momentum
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

deeper_model_1.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [87]:
dnn_history = deeper_model_1.fit(
    X_train_prepared,
    y_train,
    epochs=100,
    validation_data=(X_val_prepared, y_val),
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2911 - accuracy: 0.8967 - val_loss: 0.2754 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 2/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2814 - accuracy: 0.8991 - val_loss: 0.2789 - val_accuracy: 0.9072 - lr: 0.0010
Epoch 3/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2791 - accuracy: 0.9014 - val_loss: 0.2783 - val_accuracy: 0.9007 - lr: 0.0010
Epoch 4/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2769 - accuracy: 0.9008 - val_loss: 0.2746 - val_accuracy: 0.9053 - lr: 0.0010
Epoch 5/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2748 - accuracy: 0.9008 - val_loss: 0.2782 - val_accuracy: 0.9077 - lr: 0.0010
Epoch 6/100
1043/1043 [==============================] - 4s 4ms/step - loss: 0.2717 - accuracy: 0.9016 - val_loss: 0.2759 - val_accuracy: 0.9075 - lr: 0.0010
Epoch 7/100
1043/1043 [=============================

In [88]:
y_pred_dnn = deeper_model_1.predict(X_test_prepared)
y_pred_dnn

array([[0.1915324 ],
       [0.4149238 ],
       [0.98910236],
       ...,
       [0.0935342 ],
       [0.00656125],
       [0.00723361]], dtype=float32)

In [89]:
accuracy_score(y_test, (y_pred_dnn>=.5))

0.896091284292304

With basic set of features (i.e. without an appropriate feature engineering) the NNets do not provide a better forecast.

### 3.6.3 DNN on Polynomial Features

In [90]:
keras.backend.clear_session()

In [91]:
deeper_model_1 = keras.models.Sequential([
    keras.layers.Dense(512, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(256, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(.2),
    keras.layers.Dense(1, activation="sigmoid")
])

In [92]:
# use keras scheduler callback
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
# use early stopping callback
early_stopping = keras.callbacks.EarlyStopping(patience=10)
# I will *not* use tensorboard

# NAdam is RMSprop with Nesterov momentum
optimizer = keras.optimizers.Nadam(lr=0.001, beta_1=0.9, beta_2=0.999)

deeper_model_1.compile(
    loss="binary_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"]
)

In [93]:
dnn_history = deeper_model_1.fit(
    X_train_final,
    y_train,
    epochs=100,
    validation_data=(X_val_final, y_val),
    callbacks=[lr_scheduler, early_stopping]
)

Epoch 1/100
1043/1043 [==============================] - 21s 21ms/step - loss: 0.2980 - accuracy: 0.8948 - val_loss: 0.2776 - val_accuracy: 0.9032 - lr: 0.0010
Epoch 2/100
1043/1043 [==============================] - 22s 21ms/step - loss: 0.2837 - accuracy: 0.8989 - val_loss: 0.2897 - val_accuracy: 0.9069 - lr: 0.0010
Epoch 3/100
1043/1043 [==============================] - 22s 21ms/step - loss: 0.2776 - accuracy: 0.9005 - val_loss: 0.2780 - val_accuracy: 0.8991 - lr: 0.0010
Epoch 4/100
1043/1043 [==============================] - 22s 21ms/step - loss: 0.2737 - accuracy: 0.9009 - val_loss: 0.2785 - val_accuracy: 0.9023 - lr: 0.0010
Epoch 5/100
1043/1043 [==============================] - 22s 21ms/step - loss: 0.2698 - accuracy: 0.9023 - val_loss: 0.2846 - val_accuracy: 0.9021 - lr: 0.0010
Epoch 6/100
1043/1043 [==============================] - 23s 22ms/step - loss: 0.2654 - accuracy: 0.9036 - val_loss: 0.2830 - val_accuracy: 0.9048 - lr: 0.0010
Epoch 7/100
1043/1043 [=================

In [94]:
y_pred_dnn = deeper_model_1.predict(X_test_final)
accuracy_score(y_test, (y_pred_dnn>=.5))

0.8924496236950716

Well... I think that feature engineering for this task requires much more attention!..

In [95]:
print(f"Time elapsed since the start of the script: {time.time() - t0}s")

Time elapsed since the start of the script: 3046.453991651535s
